In [ ]:
"""
Created on Mon Jul 19 17:26:18 2021

@author: 4ka
"""
from nnalgs import simplecutloop
from ml2grb.nnalgs import reluPart, reluOBBT, prop
from ml2grb.pytorch2grb import Sequential2Grb
import sys
import numpy as np

import torch

import gurobipy as gp
from gurobipy import GRB

In [ ]:
# import my functions
%load_ext autoreload
%autoreload 2
sys.path.append('../../')

In [ ]:
# Load data
X = torch.from_numpy(np.genfromtxt('X.csv')).float()
Y = torch.from_numpy(np.genfromtxt('Y.csv')).float()

In [ ]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return X.shape[0]

    def __getitem__(self, i):
        return self.X[i, :], self.y[i]

In [ ]:
hs = 128
torch.manual_seed(10)
# Define a simple sequential network
model = torch.nn.Sequential(
    torch.nn.Linear(X.shape[1], hs),
    torch.nn.ReLU(),
    torch.nn.Linear(hs, hs),
    torch.nn.ReLU(),
    torch.nn.Linear(hs, hs),
    torch.nn.ReLU(),
    torch.nn.Linear(hs, Y.shape[1]),
)

In [ ]:
# Construct our loss function and an Optimizer.
criterion = torch.nn.MSELoss()
dataloader = torch.utils.data.DataLoader(
    MyData(X, Y), batch_size=100, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
for t in range(100):
    size = len(dataloader.dataset)
    for batch, (feat, target) in enumerate(dataloader):
        # Zero gradients
        optimizer.zero_grad()
        # Forward pass: Compute predicted y by passing x to the model
        pred = model(feat)
        # Compute and print loss
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        target = model(X)
        # Compute and print loss
        loss = criterion(target, Y)
        print(f"Epoch {t} loss: {loss.item():>7f}")

In [ ]:
p = np.random.randint(30, size=24)

In [ ]:
m = gp.Model()
x = m.addMVar((1, 48), vtype=GRB.CONTINUOUS, name="x", lb=0, ub=410)
y = m.addMVar((1, 24), lb=20, ub=30, vtype=GRB.CONTINUOUS, name="y")
m.setObjective(p @ x[0, 24:], GRB.MINIMIZE)
# Add constraint to predict value of y using x
nn2grb = Sequential2Grb(model, m)
nn2grb.predict(x, y)

In [ ]:
prediction = model.forward(X)
feasibles = X[((prediction >= 20) & (prediction <= 30)).all(axis=1), :]
sortedinputs = np.argsort(nn2grb._layers[0].invar.Obj@feasibles.numpy().T)

prop(nn2grb, feasibles[sortedinputs[0, 0]].numpy().reshape(1, -1), reset=False)

In [ ]:
for v in nn2grb.canrelax:
    v.LB = 0.0
    v.UB = 1.0
m.update()

In [ ]:
m.optimize()

In [ ]:
xin = x.X

In [ ]:
y.X

In [ ]:
for layer in nn2grb._layers:
    layer.zvar.UB = 1.0
    layer.zvar.LB = 0.0

In [ ]:
prop(nn2grb, xin, reset=False)

In [ ]:
nn2grb.obbt(2, reluOBBT('either'))

In [ ]:
m.Params.MIRCuts = 2
m.optimize()

In [ ]:
m.write('att.attr')

In [ ]:
VType = m.VType
m.setAttr(gp.GRB.Attr.VType, m.getVars(), [gp.GRB.CONTINUOUS]*len(VType))

In [ ]:
m.optimize()

In [ ]:
simplecutloop(m)

In [ ]:
xin = x.X

In [ ]:
m.optimize()